In [3]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import config
import os


## Load Files

In [14]:
train_filename = os.path.join(config.base_path, "data","训练.xlsx")
test_filename = os.path.join(config.base_path, "data","测试A.xlsx")
train_raw = pd.read_excel(train_filename) #训练数据
test_raw = pd.read_excel(test_filename) #测试数据
train_data = train_raw.iloc[:,1:] 
train_data.index = train_raw.iloc[:,0].values
test_data = test_raw.iloc[:,1:]
test_data.index = test_raw.iloc[:,0].values

print("---number of train sample---")
print(train_data.shape[0])
print("---number of test sample---")
print(test_data.shape[0])
print("---number of features---")
print(test_data.shape[1])

---number of train sample---
500
---number of test sample---
100
---number of features---
8027


## 这里人工筛了一些特征

In [15]:
drop_list = ["312X" + str(i) for i in range(1, 51)] #这几个列和TOOL(#1)这一列冗余了
drop_list += ["340X" + str(i) for i in range(1,11)] #这几个列和tool这一列冗余了
print(drop_list)
train_data = train_data.drop(columns = drop_list)
test_data = test_data.drop(columns = drop_list)

['312X1', '312X2', '312X3', '312X4', '312X5', '312X6', '312X7', '312X8', '312X9', '312X10', '312X11', '312X12', '312X13', '312X14', '312X15', '312X16', '312X17', '312X18', '312X19', '312X20', '312X21', '312X22', '312X23', '312X24', '312X25', '312X26', '312X27', '312X28', '312X29', '312X30', '312X31', '312X32', '312X33', '312X34', '312X35', '312X36', '312X37', '312X38', '312X39', '312X40', '312X41', '312X42', '312X43', '312X44', '312X45', '312X46', '312X47', '312X48', '312X49', '312X50', '340X1', '340X2', '340X3', '340X4', '340X5', '340X6', '340X7', '340X8', '340X9', '340X10']


In [16]:
cols = list(test_data.columns)
cols_tool = list(filter(lambda x : "tool" in x.lower(), cols))
print(cols_tool)

['TOOL_ID', 'Tool', 'TOOL_ID (#1)', 'TOOL_ID (#2)', 'TOOL_ID (#3)', 'Tool (#1)', 'Tool (#2)', 'tool', 'tool (#1)', 'TOOL', 'TOOL (#1)', 'Tool (#3)', 'TOOL (#2)']


## fillna with mean

In [24]:
# mean_col = train_data.mean()
# train_data_fillna = train_data.fillna(mean_col)
# mean_col = test_data.mean()
# test_data_fillna = test_data.fillna(mean_col)
# np_all = np.vstack((train_data.values[:,:-1], test_data.values))
# df_all = pd.DataFrame(np_all, columns = test_data.columns)
for i in range(len(cols_tool) - 1):
    df_tmp = train_data.loc[:,cols_tool[i]:cols_tool[i+1]].iloc[:,:-1]
    tools = df_tmp[cols_tool[i]]
    tool_set = list(set(tools))
    for k, tool in enumerate(tool_set):
        df_tool = df_tmp.loc[df_tmp[cols_tool[i]]==tool]
        mean_tool = df_tool.mean()
        df_tool = df_tool.fillna(mean_tool)
        print("--%s: %s done--" %(cols_tool[i], tool))
        if k == 0:
            train_tmp = df_tool.values
        else:
            train_tmp = np.vstack((train_tmp, df_tool.values))
    if i == 0:
        tool_frame = train_tmp
    else:
        tool_frame = np.hstack((tool_frame, train_tmp))
        
df_tmp = train_data.loc[:,cols_tool[-1]:]
tools = df_tmp[cols_tool[-1]]
tool_set = list(set(tools))
for k, tool in enumerate(tool_set):
    df_tool = df_tmp.loc[df_tmp[cols_tool[-1]]==tool]
    mean_tool = df_tool.mean()
    df_tool = df_tool.fillna(mean_tool)
    print("--%s: %s done--" %(cols_tool[i], tool))
    if k == 0:
        train_tmp = df_tool.values
    else:
        train_tmp = np.vstack((train_tmp, df_tool.values))
        
tool_frame = np.hstack((tool_frame, train_tmp))
train_data_fillna = pd.DataFrame(tool_frame, columns = train_data.columns)
    
# train_data_fillna = df_all[:train_data.shape[0]]
# test_data_fillna = df_all[train_data.shape[0] + 1:]
print(train_data_fillna["312X217"])



# np_all = np.vstack((train_data.values[:,:-1], test_data.values))
# df_all = pd.DataFrame(np_all, columns = test_data.columns)
# for i in range(len(cols_tool) - 1):
#     df_tmp = df_all.loc[:,cols_tool[i]:cols_tool[i+1]].iloc[:,:-1]
#     tools = df_tmp[cols_tool[i]]
#     tool_set = list(set(tools))
#     for k, tool in enumerate(tool_set):
#         df_tool = df_tmp.loc[df_tmp[cols_tool[i]]==tool]
#         mean_tool = df_tool.mean()
#         df_tool.fillna(mean_tool, inplace = True)
#         print("--%s: %s done--" %(cols_tool[i], tool))

        
# df_tmp = df_all.loc[:,cols_tool[-1]:]
# tools = df_tmp[cols_tool[-1]]
# tool_set = set(tools)
# for tool in tool_set:
#     df_tool = df_tmp.loc[df_tmp[cols_tool[-1]]==tool]
#     mean_tool = df_tool.mean()
#     df_tool = df_tool.fillna(mean_tool, inplace = True)
#     print("--%s: %s done--" %(cols_tool[i], tool))
    
# train_data_fillna = df_all[:train_data.shape[0]]
# test_data_fillna = df_all[train_data.shape[0] + 1:]
# print(train_data_fillna["312X217"])

--TOOL_ID: K done--
--TOOL_ID: O done--
--TOOL_ID: M done--
--TOOL_ID: J done--
--TOOL_ID: N done--
--TOOL_ID: L done--
--Tool: B done--
--Tool: A done--
--TOOL_ID (#1): E done--
--TOOL_ID (#1): N done--
--TOOL_ID (#2): E done--
--TOOL_ID (#2): D done--
--TOOL_ID (#2): C done--
--TOOL_ID (#3): N0 done--
--TOOL_ID (#3): E0 done--
--Tool (#1): 2823 done--
--Tool (#1): 329 done--
--Tool (#1): 530 done--
--Tool (#1): 1110 done--
--Tool (#1): 215 done--
--Tool (#1): 1113 done--
--Tool (#1): 1018 done--
--Tool (#1): 1245 done--
--Tool (#2): C done--
--Tool (#2): B done--
--Tool (#2): A done--
--tool: 2409 done--
--tool: 3009 done--
--tool: 4106 done--
--tool: 4147 done--
--tool (#1): U done--
--tool (#1): V done--
--tool (#1): Q done--
--tool (#1): X done--
--tool (#1): R done--
--tool (#1): P done--
--tool (#1): W done--
--tool (#1): S done--
--tool (#1): T done--
--TOOL: D done--
--TOOL: C done--
--TOOL: B done--
--TOOL (#1): YX1 done--
--TOOL (#1): XY1 done--
--Tool (#3): 6 done--
--Tool 

In [29]:
print(train_data_fillna["Tool (#1)"])

0      2823
1      2823
2      2823
3      2823
4      2823
5      2823
6      2823
7      2823
8      2823
9      2823
10     2823
11     2823
12     2823
13     2823
14     2823
15     2823
16     2823
17     2823
18     2823
19     2823
20     2823
21     2823
22     2823
23     2823
24     2823
25     2823
26     2823
27     2823
28     2823
29     2823
       ... 
470    1245
471    1245
472    1245
473    1245
474    1245
475    1245
476    1245
477    1245
478    1245
479    1245
480    1245
481    1245
482    1245
483    1245
484    1245
485    1245
486    1245
487    1245
488    1245
489    1245
490    1245
491    1245
492    1245
493    1245
494    1245
495    1245
496    1245
497    1245
498    1245
499    1245
Name: Tool (#1), Length: 500, dtype: object


## onehot编码，删除一些重复的特征和时间戳

In [7]:
selected_features = [] #保留的特征
dropped_features = [] #删除的特征
for col in test_data_fillna.columns:
    # 编码
    x = np.hstack((train_data_fillna[col].values, test_data_fillna[col].values))
    x_set_list = list(set(x))
    # if col == '210X24':
    #     print(x_set_list[0] / 10000000000)
    #     exit()
    if isinstance(x_set_list[0], str):
        encode_dict = {}
        for i in range(len(x_set_list)):
            encode_dict[x_set_list[i]] = i
        c_train = train_data_fillna[col]
        for i in range(len(c_train)):
            c_train.iat[i] = encode_dict[c_train.iat[i]]
        c_test = test_data_fillna[col]
        for i in range(len(c_test)):
            c_test.iat[i] = encode_dict[c_test.iat[i]]
    
    #去重和去时间戳
    x = train_data_fillna[col].values
    x_set_list = list(set(x))

    if len(x_set_list) >= 2 and not all([True if str(n) == "nan" else False for n in x]):
        # if col == "210X24":
        #     print(x_set_list[:20])
        #     print(str(x_set_list[0]))
        #     print([str(e).startswith("2017") or str(e).startswith("2016") for e in x_set_list[:20]])
        if col == "520X171":
            dropped_features.append(col)
        elif not all([str(e).startswith("2017") or str(e).startswith("2016") for e in x_set_list[:20]]):
            selected_features.append(col)
        else:
            dropped_features.append(col)
    else:
        dropped_features.append(col)
df_train = train_data_fillna.loc[:, selected_features + ['Y']]
df_test = test_data_fillna.loc[:, selected_features]

print("---number of features---")
print(len(selected_features))

---number of features---
6820


## 按照相关系数筛选

In [8]:
corr_values = []
k = 3000
for col in df_test.columns:
    corr_values.append(abs(pearsonr(df_train[col].values,df_train['Y'])[0]))
corr_df = pd.DataFrame({'col':df_test.columns, 'corr_value':corr_values})
corr_df = corr_df.sort_values(by='corr_value',ascending=False)
selected = corr_df['col'].values[:k]

df_train_corr = df_train.loc[:, list(selected) + ['Y']]
df_test_corr = df_test.loc[:, list(selected)]

## 按照树模型筛选

In [9]:
k = 100
X_train = df_train.values[:, 0:-1]
Y_train = df_train.values[:,-1]
X_test = df_test.values[:,:]
# x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, random_state = 1024, test_size=0.1)

# reg = GradientBoostingRegressor(random_state = 0,
#                                 learning_rate = 0.1,
#                                 n_estimators= 29,
#                                 min_samples_split=2,
#                                 min_samples_leaf=5,
#                                 max_features=0.81,
#                                 subsample= 0.8,
#                                 max_depth= 6,
# )
reg = XGBRegressor(random_state = 0,
                    learning_rate = 0.1,
                    n_estimators= 49,
                    subsample= 0.78,
                    colsample_bytree= 0.62,
                    max_depth= 3,
)
reg.fit(X_train, Y_train)
sorted_imp = sorted(reg.feature_importances_, reverse=True)
k = sorted_imp.index(0) - 1
print("---Xgboost feature importance: %dth---" %k)
print(sorted_imp[k])
importance_df = pd.DataFrame({'col':df_test.columns, 'importance':reg.feature_importances_})
importance_df = importance_df.sort_values(by="importance", ascending=False)
selected = importance_df['col'].values[:k + 1]
df_train_tree = df_train.loc[:, list(selected) + ['Y']]
df_test_tree = df_test.loc[:, list(selected)]

---Xgboost feature importance: 238th---
0.00363636


In [10]:
# after_file = os.path.join(config.base_path, "data", "tree_feature_selected_xgboost_A.xlsx")
# with pd.ExcelWriter(after_file) as writer:
#     df_train_tree.to_excel(writer,sheet_name = "train_data")
#     df_test_tree.to_excel(writer, sheet_name = "test_data")
    
after_file = os.path.join(config.base_path, "data", "feature_selected_3000_A.xlsx")
with pd.ExcelWriter(after_file) as writer:
    df_train.to_excel(writer,sheet_name = "train_data")
    df_test.to_excel(writer, sheet_name = "test_data")